In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import torch
from transformers import DPTImageProcessor, DPTForDepthEstimation
from ultralytics import YOLO
from PIL import Image
import time
from dataclasses import dataclass
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')


In [ ]:
@dataclass
class DetectedObject:
    """Data class for storing detected object information with distance"""
    class_name: str
    confidence: float
    bbox: Tuple[int, int, int, int]  # x1, y1, x2, y2
    center_point: Tuple[int, int]
    distance: float  # in meters
    min_distance: float  # minimum distance in bounding box
    max_distance: float  # maximum distance in bounding box
    avg_distance: float  # average distance in bounding box
    
    def __str__(self):
        return f"{self.class_name} (conf: {self.confidence:.2f}, dist: {self.distance:.2f}m)"


In [ ]:
class ObjectDistanceAnalyzer:
    """Combines YOLO object detection with depth estimation for distance analysis"""
    
    def __init__(self, yolo_model="yolo11n.pt", depth_model="Intel/dpt-large"):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🔧 Using device: {self.device}")
        
        # Load YOLO model
        print("📥 Loading YOLO object detection model...")
        self.yolo_model = YOLO(yolo_model)
        print("✅ YOLO model loaded successfully!")
        
        # Load depth estimation model
        print("📥 Loading depth estimation model...")
        try:
            self.depth_processor = DPTImageProcessor.from_pretrained(depth_model)
            self.depth_model = DPTForDepthEstimation.from_pretrained(depth_model)
            self.depth_model.to(self.device)
            self.depth_model.eval()
            print("✅ Depth model loaded successfully!")
        except Exception as e:
            print(f"❌ Failed to load depth model: {str(e)}")
            print("🔄 Trying fallback model...")
            self.depth_processor = DPTImageProcessor.from_pretrained("Intel/dpt-hybrid-midas")
            self.depth_model = DPTForDepthEstimation.from_pretrained("Intel/dpt-hybrid-midas")
            self.depth_model.to(self.device)
            self.depth_model.eval()
            print("✅ Fallback depth model loaded successfully!")
    
    def predict_depth(self, image):
        """Predict depth map from image"""
        try:
            # Convert to PIL Image if needed
            if isinstance(image, np.ndarray):
                pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            else:
                pil_image = image
            
            # Process image
            inputs = self.depth_processor(images=pil_image, return_tensors="pt").to(self.device)
            
            # Predict depth
            with torch.no_grad():
                outputs = self.depth_model(**inputs)
                predicted_depth = outputs.predicted_depth
            
            # Convert to numpy and normalize
            depth_map = predicted_depth.squeeze().cpu().numpy()
            
            return depth_map
            
        except Exception as e:
            print(f"Error in depth prediction: {e}")
            return None
    
    def calibrate_depth_to_distance(self, depth_map, min_distance=0.3, max_distance=10.0):
        """Convert relative depth values to real-world distances"""
        if depth_map.min() < 0:
            depth_map = -depth_map
        
        # Normalize to 0-1 range
        depth_normalized = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min())
        
        # Map to distance range (closer objects have higher depth values)
        distance_map = min_distance + (1 - depth_normalized) * (max_distance - min_distance)
        
        return distance_map
    
    def detect_objects(self, image, confidence_threshold=0.5):
        """Detect objects using YOLO"""
        results = self.yolo_model(image, conf=confidence_threshold)
        return results[0] if results else None


In [ ]:
# Additional methods for the ObjectDistanceAnalyzer class

def analyze_image(self, image, confidence_threshold=0.5):
    """Analyze image for objects and their distances"""
    # Detect objects
    detection_results = self.detect_objects(image, confidence_threshold)
    
    # Predict depth
    depth_map = self.predict_depth(image)
    if depth_map is None:
        return [], None
    
    # Convert depth to distance
    distance_map = self.calibrate_depth_to_distance(depth_map)
    
    # Resize distance map to match image dimensions
    img_height, img_width = image.shape[:2]
    distance_map_resized = cv2.resize(distance_map, (img_width, img_height))
    
    detected_objects = []
    
    if detection_results.boxes is not None:
        for box in detection_results.boxes:
            # Get bounding box coordinates
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            confidence = box.conf[0].cpu().numpy()
            class_id = int(box.cls[0].cpu().numpy())
            class_name = self.yolo_model.names[class_id]
            
            # Calculate center point
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            
            # Extract distance information from bounding box region
            bbox_distances = distance_map_resized[y1:y2, x1:x2]
            
            if bbox_distances.size > 0:
                min_distance = float(np.min(bbox_distances))
                max_distance = float(np.max(bbox_distances))
                avg_distance = float(np.mean(bbox_distances))
                center_distance = float(distance_map_resized[center_y, center_x])
            else:
                min_distance = max_distance = avg_distance = center_distance = 0.0
            
            detected_obj = DetectedObject(
                class_name=class_name,
                confidence=float(confidence),
                bbox=(x1, y1, x2, y2),
                center_point=(center_x, center_y),
                distance=center_distance,
                min_distance=min_distance,
                max_distance=max_distance,
                avg_distance=avg_distance
            )
            
            detected_objects.append(detected_obj)
    
    return detected_objects, distance_map_resized

# Monkey patch the method to the class (for notebook demonstration)
ObjectDistanceAnalyzer.analyze_image = analyze_image


In [ ]:
# Visualization method for the ObjectDistanceAnalyzer class

def visualize_results(self, image, detected_objects, distance_map=None):
    """Visualize detection results with distance information"""
    fig, axes = plt.subplots(1, 3 if distance_map is not None else 2, figsize=(20, 8))
    
    # Original image with detections
    img_with_detections = image.copy()
    
    for obj in detected_objects:
        x1, y1, x2, y2 = obj.bbox
        
        # Draw bounding box
        cv2.rectangle(img_with_detections, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Prepare label text
        label = f"{obj.class_name} {obj.confidence:.2f}\n{obj.distance:.2f}m"
        
        # Draw label background
        label_size = cv2.getTextSize(label.split('\n')[0], cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
        cv2.rectangle(img_with_detections, (x1, y1-40), (x1 + label_size[0] + 50, y1), (0, 255, 0), -1)
        
        # Draw label text
        cv2.putText(img_with_detections, obj.class_name, (x1+5, y1-25), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
        cv2.putText(img_with_detections, f"{obj.distance:.2f}m", (x1+5, y1-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
        
        # Draw center point
        cv2.circle(img_with_detections, obj.center_point, 3, (255, 0, 0), -1)
    
    # Plot 1: Original image with detections
    axes[0].imshow(cv2.cvtColor(img_with_detections, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Objects with Distance Information')
    axes[0].axis('off')
    
    # Plot 2: Distance map or statistics
    if distance_map is not None:
        im = axes[1].imshow(distance_map, cmap='jet', vmin=0, vmax=10)
        axes[1].set_title('Distance Map (meters)')
        axes[1].axis('off')
        plt.colorbar(im, ax=axes[1], shrink=0.6)
        
        # Plot 3: Overlay
        axes[2].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), alpha=0.7)
        axes[2].imshow(distance_map, cmap='jet', alpha=0.3, vmin=0, vmax=10)
        
        # Add bounding boxes to overlay
        for obj in detected_objects:
            x1, y1, x2, y2 = obj.bbox
            rect = Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, 
                           edgecolor='white', facecolor='none')
            axes[2].add_patch(rect)
            axes[2].text(x1, y1-5, f"{obj.class_name}\n{obj.distance:.2f}m", 
                       color='white', fontsize=8, weight='bold')
        
        axes[2].set_title('Distance Overlay')
        axes[2].axis('off')
    else:
        # Plot statistics if no distance map
        if detected_objects:
            distances = [obj.distance for obj in detected_objects]
            class_names = [obj.class_name for obj in detected_objects]
            
            axes[1].bar(range(len(distances)), distances)
            axes[1].set_xticks(range(len(distances)))
            axes[1].set_xticklabels(class_names, rotation=45)
            axes[1].set_ylabel('Distance (meters)')
            axes[1].set_title('Object Distances')
        else:
            axes[1].text(0.5, 0.5, 'No objects detected', 
                       ha='center', va='center', transform=axes[1].transAxes)
            axes[1].set_title('No Detections')
    
    plt.tight_layout()
    plt.show()
    
    # Print detection summary
    print("\n🎯 Detection Summary:")
    print("-" * 50)
    for i, obj in enumerate(detected_objects, 1):
        print(f"{i}. {obj}")
        print(f"   📍 Position: {obj.center_point}")
        print(f"   📏 Distance range: {obj.min_distance:.2f}m - {obj.max_distance:.2f}m (avg: {obj.avg_distance:.2f}m)")
        print()
    
    if not detected_objects:
        print("No objects detected in the image.")

# Monkey patch the visualization method
ObjectDistanceAnalyzer.visualize_results = visualize_results


In [ ]:
# Initialize the analyzer
analyzer = ObjectDistanceAnalyzer()


In [ ]:
# Capture image from webcam
cap = cv2.VideoCapture(0)

if cap.isOpened():
    ret, frame = cap.read()
    if ret:
        print("📸 Image captured from webcam")
        
        # Analyze the frame
        detected_objects, distance_map = analyzer.analyze_image(frame, confidence_threshold=0.3)
        
        # Visualize results
        analyzer.visualize_results(frame, detected_objects, distance_map)
    else:
        print("❌ Failed to capture image from webcam")
else:
    print("❌ Could not open webcam")

cap.release()


In [ ]:
def real_time_analysis(duration_seconds=30):
    """Run real-time object distance analysis for specified duration"""
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("❌ Could not open webcam")
        return
    
    print(f"🎥 Starting real-time analysis for {duration_seconds} seconds...")
    print("Press 'q' to quit early")
    
    start_time = time.time()
    frame_count = 0
    
    while time.time() - start_time < duration_seconds:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Analyze every 10th frame to maintain performance
        if frame_count % 10 == 0:
            detected_objects, _ = analyzer.analyze_image(frame, confidence_threshold=0.5)
            
            # Draw detections on frame
            for obj in detected_objects:
                x1, y1, x2, y2 = obj.bbox
                
                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # Draw label
                label = f"{obj.class_name}: {obj.distance:.1f}m"
                cv2.rectangle(frame, (x1, y1-30), (x1 + 200, y1), (0, 255, 0), -1)
                cv2.putText(frame, label, (x1+5, y1-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        
        # Display frame
        cv2.imshow('Object Distance Analysis', frame)
        
        # Check for quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print("🏁 Real-time analysis completed")

# Uncomment to run real-time analysis
# real_time_analysis(30)
